In [ ]:
from pathlib import Path
import mne
import hcp
import hcp.preprocessing as preproc
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import os
from warnings import warn
from glhcp.tools import _input_checks_meg

In [ ]:
def extract_activation_meg(
        subject,
        hcp_path,
        freq_bands=(0.5, 60),
        source=True,
        subjects_dir=None,
        parc=["aparc", "aparc.s2009a"],
        snr=1.0,
        saving_dir=None,
        create_report=True,
        verbose="ERROR"
        ):
    
    """ Preprocessing of the raw eeg recordings.
        The process could be fully or semi automatic based on user choice.

        Parameters
        ----------
        subject_id : str
            The subject name, if subject has MRI data as well, should be FreeSurfer subject name, 
            then data from both modality can be analyzed at once.
        subjects_dir : path-like | None
            The path to the directory containing the EEG subjects. The folder structure should be
            as following which can be created by running "file_preparation" function:
            subjects_dir/
            ├── sMRI/
            ├── fMRI/
            │   ├── session_1/
            │   ├── session_2/
            ├── dMRI/
            ├── EEG/
                ├── paradigm_1/
                ├── paradigm_2/
                ├── ...
        site : str
            The recording site; must be one of the following: ["Austin", "Dublin", "Ghent", "Illinois", "Regensburg", "Tuebingen"]
        paradigm : str
            Name of the EEG paradigm. should be a subfolder in the subjects_dir / subject_id containing
            raw EEG data.
        psd_check : bool
            if True, the psd will be shown, by clicking on noisy channels, you can see the bad channel names.
        manual_data_scroll : bool
            If True, user can interactively annotate segments of the recording to be removed.
            If not, this step will be skipped.
        run_ica: bool
            If True, ICA will be perfomed to detect and remove eye movement components from data.
            This option is set for the gpias paradigm.
        manual_ica_removal : bool
            If True, a window will pop up asking ICA components to be removed.
            If not, a machine learning model will be used to remove ICA components related to eye movements.
        ssp_eog : bool
            If True, will use EOG channels to regress out blinking from data.
        ssp_ecg : bool
            If True, will use ECG channels to regress out ECG artifact from data.
        create_report : bool
            If True, a report will be created per recordinng.
        saving_dir : path-like | None | bool
            The path to the directory where the preprocessed EEG will be saved, If None, it will be saved 
            in the same path as the raw files. If False, preprocessed data will not be saved.
        verbose : bool | str | int | None
            Control verbosity of the logging output. If None, use the default verbosity level.

        Notes
        -----
        .. This script is mainly designed for Antinomics / TIDE projects, however could be 
            used for other purposes.
        """
    
    mne.set_log_level(verbose)

    ## add progress bar here

    ## input checks
    hcp_path, subjects_dir, freqs, parc, saving_dir = \
        _input_checks_meg(subject,
                            hcp_path,
                            freq_band,
                            source,
                            subjects_dir,
                            parc,
                            snr,
                            saving_dir,
                            create_report
                            )

    hcp_path_proc = hcp_path / "processed"
    hcp_path_unproc = hcp_path / "unprocessed"
    hcp_head_mri = hcp_path / "trans"

    ## add checks for these paths to exist

    kwargs_unproc = {
                    "subject": subject,
                    "run_index": 1,
                    "data_type": "rest",
                    "hcp_path": hcp_path / "unprocessed"
                    }
    kwargs_proc = {
                    "subject": subject,
                    "run_index": 1,
                    "data_type": "rest",
                    "hcp_path": hcp_path / "processed"
                    }
    
    ## read some necessary information
    info = hcp.read_info(**kwargs_unproc)
    annots = hcp.read_annot(**kwargs_proc)
    ica = hcp.read_ica(**kwargs_proc)
    raw = hcp.read_raw(**kwargs_unproc)
    raw.load_data()
    
    ## start preprocessing

    # add resampling
    
    l_freq, h_freq = 0.1, 80
    preproc.apply_ref_correction(raw)
    bad_seg = (annots['segments']['all']) / raw.info['sfreq']
    annotations = mne.Annotations(
                                bad_seg[:, 0], (bad_seg[:, 1] - bad_seg[:, 0]),
                                description='bad'
                                )
    raw.set_annotations(annotations)
    raw.info['bads'].extend(annots['channels']['all'])
    raw.pick_types(meg=True, ref_meg=False)

    ## filter and ica
    raw.filter(l_freq, h_freq, method='iir', iir_params=dict(order=4, ftype='butter'), verbose=False)
    exclude = [ii for ii in range(annots['ica']['total_ic_number'][0]) if ii not in annots['ica']['brain_ic_vs']]
    preproc.apply_ica_hcp(raw, ica_mat=ica, exclude=exclude)
    
    
    
    if not source:
        return x # a dictionary with data and ch_names

    ## handling empty room noise
    raw_noise = hcp.read_raw(subject=subject,
                            hcp_path=hcp_path,
                            data_type='noise_empty_room'
                            )
    raw_noise.filter(l_freq, h_freq, method='iir', iir_params=dict(order=4, ftype='butter'))
    noise_cov = mne.compute_raw_covariance(raw_noise, method='empirical')
    del raw_noise

    ## create subjects dir
    hcp.make_mne_anatomy(subject=subject,
                        subjects_dir=subjects_dir,
                        hcp_path=hcp_path,
                        recordings_path=hcp_head_mri,
                        outputs=('label', 'mri', 'surf')
                        )
    
    ## setup source space
    src = mne.setup_source_space(subject=subject,
                                    subjects_dir=subjects_dir,
                                    add_dist=True,
                                    spacing='oct6'
                                    )

    head_mri_t = mne.read_trans(fname=hcp_head_mri / f"{subject}" /f"{subject}-head_mri-trans.fif")
    bems = mne.make_bem_model(subject,
                                conductivity=(0.3,),
                                subjects_dir=subjects_dir,
                                ico=None
                                )
    bem_sol = mne.make_bem_solution(bems)
    bem_sol['surfs'][0]['coord_frame'] = 5

    picks = mne.pick_types(raw.info, meg=True, ref_meg=False)
    info = mne.pick_info(raw.info, picks)

    fwd = mne.make_forward_solution(info,
                                    src=src,
                                    trans=head_mri_t,
                                    bem=bem_sol
                                    )
    
    inv_op = mne.minimum_norm.make_inverse_operator(info,
                                                fwd,
                                                noise_cov
                                                )
    stcs = mne.minimum_norm.apply_inverse_raw(raw,
                                        inv_op,
                                        lambda2=1.0 / snr**2
                                        )

    ## extract signals from parcels
    
    extraction_mode = "mean" # maybe go as an input to function
    
    for par in parc:
        labels = mne.read_labels_from_annot(subject,
                                            parc=parc,
                                            subjects_dir=subjects_dir
                                            )
        label_ts = mne.extract_label_time_course(stcs,
                                                labels,
                                                src,
                                                allow_empty=True,
                                                mode=extraction_mode,
                                                return_generator=True
                                                )


